Con este notebook se obtiene una lista con las compañías que se van a usar en el modelo

In [156]:
import requests
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [157]:
key_finnhub = "your_finnhub_api_key_here"

In [158]:
symbols = list()

In [159]:
market = "MC"
url = f"https://finnhub.io/api/v1/stock/symbol?exchange={market}&token={key_finnhub}"
r = requests.get(url)
l = r.json()
all([elem.get('symbol') == elem.get('displaySymbol') for elem in l]), len(l)

(True, 204)

In [160]:
symbols.extend(l)

In [161]:
market = "L"
url = f"https://finnhub.io/api/v1/stock/symbol?exchange={market}&token={key_finnhub}"
r = requests.get(url)
l = r.json()
all([elem.get('symbol') == elem.get('displaySymbol') for elem in l]), len(l)

(True, 4094)

In [162]:
symbols.extend(l)

In [163]:
market = "US"
url = f"https://finnhub.io/api/v1/stock/symbol?exchange={market}&token={key_finnhub}"
r = requests.get(url)
l = r.json()
all([elem.get('symbol') == elem.get('displaySymbol') for elem in l]), len(l)

(True, 8930)

In [164]:
symbols.extend(l)

In [213]:
df = pd.DataFrame(symbols)

In [214]:
df['description'].value_counts()

                                         1020
ISHARES IV PLC                            129
MULTI UNITS LUXEMBOURG                    124
XTRACKERS                                 104
ISHARES II PLC                             86
                                         ... 
BLACKROCK MUNIYIELD ARIZONA                 1
ENERGIZER HOLDINGS INC                      1
Autonomy Spain Real Estate SOCIMI SAU       1
CALAMOS GLOBAL DYNAMIC INCOM                1
EP GLOBAL OPPORTUNITIES TRUST PLC           1
Name: description, Length: 10030, dtype: int64

In [218]:
# Me quedo con los que tienen descripcion unica
df_tot = df[~df['description'].duplicated(keep=False)]

In [219]:
df_tot['description'].value_counts()

O'SHARES GLOBAL INTERNET GIA         1
OILEX LD                             1
Aperam SA                            1
GETECH GROUP PLC                     1
JOHN HANCOCK MULT FACT INDUS         1
                                    ..
VERASTEM INC                         1
BARONSMEAD VENTURE TRUST PLC         1
IVERIC BIO INC                       1
TFI INTERNATIONAL INC                1
EP GLOBAL OPPORTUNITIES TRUST PLC    1
Name: description, Length: 9682, dtype: int64

In [220]:
df_tot['p1'] = df_tot['symbol'].str.split('.').str[0]

//anaconda3/envs/bsm03/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [211]:
df_tot[df_tot['symbol'].str.startswith('SAN')]

,description,displaySymbol,symbol,p1
142,Banco Santander SA,SAN.MC,SAN.MC,SAN
11282,BANCO SANTANDER SA-SPON ADR,SAN,SAN,SAN
11284,SANDSTORM GOLD LTD,SAND,SAND,SAND
11285,SANMINA CORP,SANM,SANM,SANM
11286,S&W SEED CO,SANW,SANW,SANW


In [223]:
df_tot['p1'].value_counts()

DIA     3
BIO     3
MED     3
COM     3
IAG     3
       ..
BWA     1
LEU     1
TSRI    1
MAW     1
STEM    1
Name: p1, Length: 9184, dtype: int64

In [170]:
dup = df_tot[df_tot['p1'].duplicated(keep=False)].sort_values(by='p1')

In [183]:
choices = dup['description'].values

In [184]:
list_ = list()
for elem in choices:
    p = process.extract(elem, choices, limit=3)
    if p[1][1] >= 90:
        list_.append({'a': p[0][0], 'b': p[1][0], 'score': p[1][1]})

In [185]:
dup

,description,displaySymbol,symbol,p1
4299,ALCOA CORP,AA,AA,AA
214,AA PLC,AA.L,AA.L,AA
4303,AMERICAN AIRLINES GROUP INC,AAL,AAL,AAL
416,ANGLO AMERICAN PLC,AAL.L,AAL.L,AAL
4312,ALMADEN MINERALS LTD - B,AAU,AAU,AAU
...,...,...,...,...
13083,PROSHARES ULTRA FTSE CHINA50,XPP,XPP,XPP
13136,YATRA ONLINE INC,YTRA,YTRA,YTRA
181,Trajano Iberia Socimi SA,YTRA.MC,YTRA.MC,YTRA
4290,ZENITH ENERGY LTD.,ZEN.L,ZEN.L,ZEN


In [186]:
df_scr = pd.DataFrame(list_)

In [187]:
dataframe = pd.merge(dup, df_scr, left_on='description', right_on='a')

In [225]:
dataframe

,description,displaySymbol,symbol,p1,a,b,score
0,FEDERAL AGRIC MTG CORP-CL A,AGM.A,AGM.A,AGM,FEDERAL AGRIC MTG CORP-CL A,FEDERAL AGRIC MTG CORP-CL C,96
1,FEDERAL AGRIC MTG CORP-CL C,AGM,AGM,AGM,FEDERAL AGRIC MTG CORP-CL C,FEDERAL AGRIC MTG CORP-CL A,96
2,EMBOTELLADORA ANDINA-ADR A,AKO.A,AKO.A,AKO,EMBOTELLADORA ANDINA-ADR A,EMBOTELLADORA ANDINA-ADR B,96
3,EMBOTELLADORA ANDINA-ADR B,AKO.B,AKO.B,AKO,EMBOTELLADORA ANDINA-ADR B,EMBOTELLADORA ANDINA-ADR A,96
4,ASTRAZENECA PLC,AZN.L,AZN.L,AZN,ASTRAZENECA PLC,ASTRAZENECA PLC-SPONS ADR,90
...,...,...,...,...,...,...,...
66,VODAFONE GROUP PLC-SP ADR,VOD,VOD,VOD,VODAFONE GROUP PLC-SP ADR,VODAFONE GROUP PLC,95
67,WPP PLC,WPP.L,WPP.L,WPP,WPP PLC,WPP PLC-SPONSORED ADR,90
68,WPP PLC-SPONSORED ADR,WPP,WPP,WPP,WPP PLC-SPONSORED ADR,WPP PLC,90
69,WATSCO INC,WSO,WSO,WSO,WATSCO INC,WATSCO INC -CL B,90


In [226]:
val = [elem for elem in dataframe['description'].values if '-' in elem]

In [227]:
dataframe[dataframe['description'].isin(val)]

,description,displaySymbol,symbol,p1,a,b,score
0,FEDERAL AGRIC MTG CORP-CL A,AGM.A,AGM.A,AGM,FEDERAL AGRIC MTG CORP-CL A,FEDERAL AGRIC MTG CORP-CL C,96
1,FEDERAL AGRIC MTG CORP-CL C,AGM,AGM,AGM,FEDERAL AGRIC MTG CORP-CL C,FEDERAL AGRIC MTG CORP-CL A,96
2,EMBOTELLADORA ANDINA-ADR A,AKO.A,AKO.A,AKO,EMBOTELLADORA ANDINA-ADR A,EMBOTELLADORA ANDINA-ADR B,96
3,EMBOTELLADORA ANDINA-ADR B,AKO.B,AKO.B,AKO,EMBOTELLADORA ANDINA-ADR B,EMBOTELLADORA ANDINA-ADR A,96
5,ASTRAZENECA PLC-SPONS ADR,AZN,AZN,AZN,ASTRAZENECA PLC-SPONS ADR,ASTRAZENECA PLC,90
7,BROWN-FORMAN CORP-CLASS B,BF.B,BF.B,BF,BROWN-FORMAN CORP-CLASS B,BROWN-FORMAN CORP-CLASS A,96
8,BROWN-FORMAN CORP-CLASS A,BF.A,BF.A,BF,BROWN-FORMAN CORP-CLASS A,BROWN-FORMAN CORP-CLASS B,96
9,BIGLARI HOLDINGS INC-B,BH,BH,BH,BIGLARI HOLDINGS INC-B,BIGLARI HOLDINGS INC-A,95
10,BIGLARI HOLDINGS INC-A,BH.A,BH.A,BH,BIGLARI HOLDINGS INC-A,BIGLARI HOLDINGS INC-B,95
11,BIO-RAD LABORATORIES-A,BIO,BIO,BIO,BIO-RAD LABORATORIES-A,BIO-RAD LABORATORIES -CL B,90


In [229]:
df_fin = df_tot[~df_tot['description'].isin(val)]

In [233]:
df_fin['p1'].value_counts()

PAC      3
BME      3
GRF      3
AMS      3
COM      3
        ..
GRP=     1
KCAC=    1
CTB      1
PAUG     1
STEM     1
Name: p1, Length: 9173, dtype: int64

In [234]:
df_fin[df_fin['p1'] == 'PAC']

,description,displaySymbol,symbol,p1
123,Papeles y Cartones de Europa SA,PAC.MC,PAC.MC,PAC
2901,PACIFIC ASSETS TRUST PLC,PAC.L,PAC.L,PAC
10286,GRUPO AEROPORTUARIO PAC-ADR,PAC,PAC,PAC


In [235]:
v = df_fin['description'].values

In [238]:
len([e for e in v if '-' not in e])

8407

In [240]:
df_fin['symbol'].value_counts()

CDE        1
LDOS       1
SINO       1
LZB        1
WIT        1
          ..
EARS       1
EVK        1
AFSM       1
IYW        1
SGRE.MC    1
Name: symbol, Length: 9641, dtype: int64

In [242]:
'AMZN' in df_fin['symbol'].values

True

In [253]:
data = df_fin[['description', 'symbol']].reset_index(drop=True)

In [254]:
data = data[~data['description'].str.endswith('USD')]

In [255]:
data = data[~data['description'].str.endswith('CLASS A')]

In [260]:
data = data.reset_index(drop=True)

In [270]:
data.

(9530, 2)

In [268]:
data[data['description'].str.endswith('- A')]

,description,symbol
2035,ASSOCIATED CAPITAL GROUP - A,AC
2038,ACAMAR PARTNERS ACQUISITI- A,ACAM
2244,ALTAIR ENGINEERING INC - A,ALTR
2271,AMERICAN HOMES 4 RENT- A,AMH
2319,ANGI HOMESERVICES INC- A,ANGI
2357,APPFOLIO INC - A,APPF
2395,ARES MANAGEMENT CORP - A,ARES
2487,ALTICE USA INC- A,ATUS
2628,ATRECA INC - A,BCEL
2661,BLOOM ENERGY CORP- A,BE


In [267]:
data[data['description'].str.startswith('MICROSOFT')]

,description,symbol
6497,MICROSOFT CORP,MSFT


In [271]:
data.to_csv('/Users/esanc147/Documents/business/bsm03/web_app/data/symbol/symbols.csv', index=False, header=False)

In [272]:
d = pd.read_csv('/Users/esanc147/Documents/business/bsm03/web_app/data/symbol/symbols.csv', names=['desc', 'symbol'])

In [279]:
d['symbol'].value_counts()

CDE        1
MAXR       1
NRZ        1
LZB        1
WIT        1
          ..
IYW        1
LBE.L      1
AAXN       1
QCRH       1
SGRE.MC    1
Name: symbol, Length: 9530, dtype: int64

In [275]:
9000/120

75.0

In [280]:
import multiprocessing as mp

In [302]:
def cube(x):
    return x**3

pool = mp.Pool(processes=2)
%timeit pool.map(cube, range(1,10000))

2.92 ms ± 64.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [303]:
%timeit [cube(e) for e in range(1, 10000)]

3.25 ms ± 41.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [306]:
f = pd.read_csv('/Users/esanc147/Documents/business/bsm03/web_app/data/symbol/failed_symbols.csv', names=['symbol'])

In [311]:
dd = d[~d['symbol'].isin(f['symbol'].values)]

In [310]:
d.shape

(9530, 2)

In [312]:
dd.shape

(9155, 2)

In [313]:
dd.to_csv('/Users/esanc147/Documents/business/bsm03/web_app/data/symbol/symbols.csv', index=False, header=False)

In [314]:
import os

In [714]:
len(os.listdir('/Users/esanc147/Documents/business/bsm03/web_app/data/tech/'))

8824

In [704]:
list_dir = os.listdir('/Users/esanc147/Documents/business/bsm03/web_app/data/tech/')

In [705]:
len(list_dir)

7287

In [706]:
ld = [elem.split('.csv')[0] for elem in list_dir]

In [423]:
fai = ['MHID.L', 'LTOD.L', 'KAKU.L', 'SPEC.L', 'INX.L', 'IDH.L', 'HNT.L', 'OTES.L', 'FPP.L', 'FP.L', 'BGX.L', 'CGO.L',
      'COBR.L', 'CNSD.L', 'SNP.L', 'CHF.L', 'CFHS.L', 'BIH.L', 'BPFU.L', 'BLBD.L', 'BRES.L']

In [622]:
len(ld)

3032

In [428]:
ld.extend(fai)

In [623]:
len(ld)

3032

In [406]:
# dd_aux.to_csv('/Users/esanc147/Documents/business/bsm03/web_app/data/symbol/aux_symbols.csv', index=False, header=False)

In [407]:
# dd.to_csv('/Users/esanc147/Documents/business/bsm03/web_app/data/symbol/symbols.csv', index=False, header=False)

In [707]:
dd_aux = dd_aux[~dd_aux['symbol'].isin(ld)]

In [708]:
dd_aux.shape

(1843, 2)

In [709]:
dd_aux.to_csv('/Users/esanc147/Documents/business/bsm03/web_app/data/symbol/aux_symbols.csv', index=False, header=False)